In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("talk")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml results_cfg 

log:
    root: /storage/vbutoi/scratch/ESE/inference
    inference_groups: 
        # - "04_11_24_WBC_UniverSeg_CalibrationAblation"
        - "04_11_24_SpineWeb_UniverSeg_CalibrationAblation"
    
calibration:
    metric_cfg_file: "/storage/vbutoi/projects/ESE/ese/experiment/configs/inference/Calibration_Metrics.yaml"

options:
    add_dice_loss_rows: True
    drop_nan_metric_rows: True 
    remove_shared_columns: False
    equal_rows_per_cfg_assert: True 

In [ ]:
from ese.experiment.analysis.analyze_inf import load_cal_inference_stats

image_info_df = load_cal_inference_stats(
    results_cfg=results_cfg,
    load_cached=False,
)

In [ ]:
image_info_df['calibrator'].unique()

In [ ]:
# Sort the image_info_df by method name, so everything appears nicely
image_info_df = image_info_df.sort_values(by=['method_name', 'calibrator'])
# Make sure that the model_class 'Uncalibrated' is first
image_info_df['calibrator'] = image_info_df['calibrator'].astype('category')
image_info_df['calibrator'] = image_info_df['calibrator'].cat.reorder_categories([
    'Uncalibrated',
])

In [ ]:
image_info_df.keys()

In [ ]:
image_info_df['image_metric'].unique()

In [ ]:
# Sort the df by the data_id
image_info_df = image_info_df.sort_values(by=['data_id'])

In [ ]:
dice_info_df = image_info_df[image_info_df['image_metric'] == 'Dice Loss']

plt.figure(figsize=(12, 8))
sns.boxplot(
    x='data_id', 
    y='metric_score', 
    data=dice_info_df
)
plt.ylabel('Dice Loss')
plt.title('SpineWeb Dice Loss per Datapoint')

In [ ]:
ece_info_df = image_info_df[image_info_df['image_metric'] == 'Image_ECE']

plt.figure(figsize=(12, 8))
sns.boxplot(
    x='data_id', 
    y='metric_score', 
    data=ece_info_df
)
# Set the y label to be Image ECE
plt.ylabel('Image ECE')
plt.title('SpineWeb Image ECE per Datapoint')